# RAG 基础架构
> By SherryAGI | Retrieval-Augmented Generation basics

## What is RAG？

**Retrieval-Augmented Generation** = Retrieval增强Generation

```
用户提问 → Retrieval相关文档 → 结合文档GenerationAnswer
```

### Why do we need RAG？

| Question | 纯 LLM | RAG |
|------|--------|-----|
| knowledge过时 | ❌ 训练data截止 | ✅ 实时Retrieval最新文档 |
| 私有data | ❌ 无法访问 | ✅ Connectionenterpriseknowledge库 |
| 幻觉Question | ❌ 可能编造 | ✅ 基于真实文档 |
| Source追溯 | ❌ 无法验证 | ✅ 引用原文出处 |

In [ ]:
# 安装依赖
# pip install langchain langchain-openai chromadb

import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings

print('🚀 RAG 基础架构 by SherryAGI')

## RAG Core Process

```
1. Document Loading (Load)
   ↓
2. Text Chunking (Split)
   ↓
3. Vectorization (Embed)
   ↓
4. Storage (Store)
   ↓
5. Retrieval (Retrieve)
   ↓
6. Generation (Generate)
```

## 1. Document Loading

In [ ]:
# Example：enterpriseknowledge文档
documents = [
    {
        "content": """Company reimbursement policy：
        1. Travel expenses：economy class tickets、3-star hotels
        2. Meal expenses：daily limit 100 CNY
        3. Transportation：public transport or taxi（receipt required）
        4. Approval process：500CNYdirect reimbursement，500CNYrequires manager approval""",
        "metadata": {"source": "HR政策", "category": "报销"}
    },
    {
        "content": """Leave policy：
        1. Annual leave：after1years entitled to5days，each additional1year adds1days，maximum15days
        2. Sick leave：requires hospital certificate，annual cumulative maximum30days
        3. Personal leave：requires advance notice of3days申请，annual cumulative maximum10days
        4. approval：3days以内主管approval，3days以上需HRapproval""",
        "metadata": {"source": "HR政策", "category": "请假"}
    },
    {
        "content": """IT Support process：
        1. Submit ticket：via internal system
        2. Response time：Normal issues24hours，Urgent issues4hours
        3. Equipment request：需部门主管approval
        4. Password reset：self-service or contact IT hotline 8888""",
        "metadata": {"source": "IT手册", "category": "支持"}
    }
]

print(f'Loaded {len(documents)} documents')

## 2. Text Chunking

In [ ]:
# 分chunks策略
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,      # 每chunks最大characters数
    chunk_overlap=50,    # chunks之间重叠characters数
    separators=["\n\n", "\n", "。", "，", " "]
)

# 分chunks
chunks = []
for doc in documents:
    splits = text_splitter.split_text(doc["content"])
    for split in splits:
        chunks.append({
            "content": split,
            "metadata": doc["metadata"]
        })

print(f'After chunking: {len(chunks)} 个Chunk')
for i, chunk in enumerate(chunks[:3]):
    print(f'\n--- Chunk {i+1} ---')
    print(chunk["content"][:100] + '...')

## 3. Vectorization & Storage

In [ ]:
# 使用 OpenAI Embeddings（需要 API Key）
# os.environ["OPENAI_API_KEY"] = "your-api-key"

# 或使用本地Models（recommendedenterprise使用）
# from langchain_community.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-zh-v1.5")

# Example：模拟Vectorization
import numpy as np

def mock_embed(text):
    """模拟Vectorization（实际使用时替换为真实 Embedding Models）"""
    np.random.seed(hash(text) % 2**32)
    return np.random.rand(384).tolist()

# Vectorization所有Chunk
for chunk in chunks:
    chunk["embedding"] = mock_embed(chunk["content"])

print(f'Vector dimension: {len(chunks[0]["embedding"])}')

## 4. Retrieval

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def search(query, top_k=3):
    """语义搜索"""
    query_embedding = mock_embed(query)
    
    # 计算Similarity
    similarities = []
    for chunk in chunks:
        sim = cosine_similarity([query_embedding], [chunk["embedding"]])[0][0]
        similarities.append((sim, chunk))
    
    # 排序返回 Top K
    similarities.sort(key=lambda x: -x[0])
    return similarities[:top_k]

# TestRetrieval
query = "What is the travel reimbursement policy？"
results = search(query)

print(f'🔍 Query: {query}\n')
for i, (score, chunk) in enumerate(results):
    print(f'--- Result {i+1} (Similarity: {score:.3f}) ---')
    print(f'Source: {chunk["metadata"]["source"]}')
    print(chunk["content"][:150])
    print()

## 5. GenerationAnswer

In [ ]:
def generate_answer(query, context_chunks):
    """基于RetrievalResultGenerationAnswer（Example）"""
    
    # 构建 Prompt
    context = "\n\n".join([c["content"] for _, c in context_chunks])
    
    prompt = f"""基于以下enterpriseknowledge库内容回答Question。如果无法从内容中找到Answer，请说明。

knowledge库内容：
{context}

Question：{query}

Answer："""
    
    # 实际使用时调用 LLM
    # response = llm.invoke(prompt)
    
    print('📝 Generation的 Prompt:')
    print(prompt)
    print('\n💡 提示：将此 Prompt 发送给 LLM 即可获得Answer')

generate_answer(query, results)

## 6. Complete RAG Pipeline

In [ ]:
class SimpleRAG:
    """简单 RAG 实现 by SherryAGI"""
    
    def __init__(self):
        self.chunks = []
    
    def add_document(self, content, metadata=None):
        """Add document"""
        chunk = {
            "content": content,
            "metadata": metadata or {},
            "embedding": mock_embed(content)
        }
        self.chunks.append(chunk)
    
    def query(self, question, top_k=3):
        """Query"""
        results = search(question, top_k)
        return {
            "question": question,
            "sources": [c["metadata"].get("source", "Unknown") for _, c in results],
            "context": [c["content"] for _, c in results]
        }

# 使用Example
rag = SimpleRAG()
for doc in documents:
    rag.add_document(doc["content"], doc["metadata"])

result = rag.query("Annual leave有多少days？")
print(f'Question: {result["question"]}')
print(f'Source: {result["sources"]}')

---
**By SherryAGI** | [DigitalTransformationAI](https://github.com/AIB612/DigitalTransformationAI)